#### Grade [radverma](https://github.iu.edu/CSCI-P556-Spring-2021/P556-radverma/tree/master/hw3) submission for hw3

## Baseline-Dropout

In [16]:
from sklearn.datasets import fetch_openml
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
import collections

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.functional as Func
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset
import random

In [18]:
from scipy.io import savemat
import os 
from os import path
from sklearn.preprocessing import normalize
from torch.nn.utils import clip_grad_norm_
import torch.nn.parallel.data_parallel as data_parallel
from sklearn.metrics import confusion_matrix
from torch.optim.lr_scheduler import LambdaLR, StepLR, MultiStepLR, ExponentialLR, ReduceLROnPlateau

In [19]:
mnist_data = fetch_openml('mnist_784',version=1)
mnist_data.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [20]:
import numpy as np
X,y=mnist_data["data"],mnist_data["target"]
y=y.astype(int)
X=((X/255.)-.5)*2
print(X.shape,y.shape)
print(min(y),max(y))
print(type(X),type(y))
print(np.min(X), np.max(X))

(70000, 784) (70000,)
0 9
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
-1.0 1.0


In [21]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(60000, 784) (10000, 784) (60000,) (10000,)


In [22]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state = 42)

for train_index, val_index in split.split(X_train,y_train):
    X_train_strat = X[train_index,:]
    y_train_strat = y[train_index]
    
    X_dev_strat = X[val_index,:]
    y_dev_strat = y[val_index]

print(X_train_strat.shape,y_train_strat.shape,X_dev_strat.shape,y_dev_strat.shape)

(54000, 784) (54000,) (6000, 784) (6000,)


In [23]:
train_images = X_train_strat.reshape(X_train_strat.shape[0], 28, 28) #reshape the images to 54000,28,28 from 54000,784
dev_images = X_dev_strat.reshape(X_dev_strat.shape[0], 28, 28)
test_images = X_test.reshape(X_test.shape[0], 28, 28)

In [24]:
#train
train_images_tensor = torch.tensor(train_images)/255.0
train_labels_tensor = torch.tensor(y_train_strat)
train_tensor = TensorDataset(train_images_tensor, train_labels_tensor)

#dev
dev_images_tensor = torch.tensor(dev_images)/255.0
dev_labels_tensor = torch.tensor(y_dev_strat)
dev_tensor = TensorDataset(dev_images_tensor, dev_labels_tensor)

#test
test_images_tensor = torch.tensor(test_images)/255.0

In [25]:
# train
train_loader = DataLoader(train_tensor, batch_size=16, num_workers=2, shuffle=True)
# dev loader
dev_loader = DataLoader(dev_tensor, batch_size=16, num_workers=2, shuffle=True)
# test loader
test_loader = DataLoader(test_images_tensor, batch_size=16, num_workers=2, shuffle=False)

In [38]:
# Define key hyperparameters
class hyperparam:
    num_hid1_units = 300
    num_hid2_units = 100
    num_classes = 10
    input_dimension = 28*28
    
    # learning rate
    lr = 0.05  
    # Number of epoch
    num_epochs = 5  
    # Mini-batch size
    bs = 16   
    # Momentum SGD 
    moment= 0.9 

In [39]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2) 
        )
        
        self.linear_block = nn.Sequential(
            #nn.Dropout(p=0.5),
            nn.Linear(128*7*7, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            #nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            #nn.Dropout(0.5),
            nn.Linear(64, 10)
        )
        
    def forwardProp(self, x):
        x = self.conv_block(x)
        x = x.view(x.size(0), -1)
        x = self.linear_block(x)
        
        return x

In [40]:
def initial_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.01)

In [41]:
# Create DNN and define loss function
cnn_model = CNN()
cnn_model.apply(initial_weights)
cnn_model

CNN(
  (conv_block): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_block): Sequential(
    (0): Linear(in_features=6272, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3

In [42]:
optimizer = torch.optim.SGD(cnn_model.parameters(), lr=hyperparam.lr, momentum=hyperparam.moment)
scheduler = StepLR(optimizer, step_size=100, gamma=0.1)
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    print('Yes, Available')
    cnn_model = cnn_model.cuda()
    criterion = criterion.cuda()

In [43]:
#Train the DNN
train_avg_loss= []
train_accuracy =[]
dev_avgLoss = []
dev_accuracy= []

#loop over epochs
for epoch in range(hyperparam.num_epochs):
    tr_num_correct = 0
    tr_num_samples = 0
    tr_total_loss = 0.0
    dev_num_correct = 0
    dev_num_samples = 0
    dev_total_loss = 0.0
    
    #Training
    cnn_model.train(True) #does each opoch initialise the entire dnn model with all the weights and stuff?
    scheduler.step()

    with torch.set_grad_enabled(True):
        for local_batch, local_labels in train_loader:
            local_batch=local_batch.unsqueeze(1)
            
            if torch.cuda.is_available():
              local_batch = local_batch.cuda()
              local_labels = local_labels.cuda()
            
            optimizer.zero_grad()
            
            
            local_batch = local_batch.float()
            local_labels = local_labels.float()
            #local_batch, local_labels = Variable(local_batch), Variable(local_labels)
            
            #Model computations
            out1 = cnn_model(local_batch)
            
            #Cross Entropy loss calculation
            ploss = criterion(out1, local_labels.long())
            tr_total_loss += ploss*hyperparam.bs
            
            #Backpropogation
            ploss.backward() #gradient calculation
            optimizer.step() #weight update
            #scheduler.step() #to decrease the lr by 10% every 10 epochs because as you move towards the minima the steps size needs to decrease so it doesnt jump over the minima
            
            sel_class = torch.argmax(out1, dim=1) #chooses the maximum value of the softmax layer
            
            tr_num_correct += sel_class.eq(local_labels).sum().item()
            tr_num_samples += hyperparam.bs
        
        tr_avg_Loss = tr_total_loss/len(train_loader.dataset)
        train_avg_loss.append(tr_avgLoss)
        
        tr_accuracy = tr_num_correct/tr_num_samples
        train_accuracy.append(tr_accuracy)

    #validation 
    with torch.set_grad_enabled(False):
      cnn_model.eval()

      for local_batch,local_labels in dev_loader:
        local_batch=local_batch.unsqueeze(1)

        local_batch=local_batch.float()
        local_labels=local_labels.float()
        
        
        if torch.cuda.is_available():
            local_batch = local_batch.cuda()
            local_labels = local_labels.cuda()
     
        out1=cnn_model(local_batch)

        #CrossEntropy 
        pLoss=criterion(out1,local_labels.long())
        dev_total_loss +=ploss*hyperparam.bs
        sel_class=torch.argmax(out1,dim=1)

        dev_num_correct+=sel_class.eq(local_labels).sum().item()

        #print(correction)
        dev_num_samples+=hyperparam.bs

      dev_avgLoss=dev_total_loss/len(dev_loader.dataset)
      dev_avgLoss.append(dev_avgLoss)

      dev_accuracy=dev_num_correct/dev_num_samples
      dev_accuracy.append(dev_accuracy)
    print("Accuracy",tr_accuracy,dev_accuracy)

# Train Accuracy
plt.plot(np.arange(0,hyperparam.num_epochs,1),tr_accuracy_list)
plt.xlabel("Epochs")
plt.ylabel("Train Accuracy")
plt.show()

# Validation
plt.plot(np.arange(0,hyperparam.num_epochs,1),dev_accuracy_list)
plt.xlabel("epochs")
plt.ylabel("Validation Accuracy")
plt.show()

# ------ Errors ---------

# Train Errors
plt.plot(np.arange(0,hyperparam.num_epochs,1),tr_avgLoss_list)
plt.xlabel("Epochs")
plt.ylabel("Train Errors")
plt.show()

# Errors
plt.plot(np.arange(0,hyperparam.num_epochs,1),dev_avgLoss_list)
plt.xlabel("Epochs")
plt.ylabel("Validation Errors")
plt.show()

C:\Users\vishw\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


NotImplementedError: 

In [37]:
def predictions(data_loader):
    cnn_model.eval()
    test_predictions = torch.LongTensor()
    
    for i, data in enumerate(data_loader):
        data = data.unsqueeze(1) 
        
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = cnn_model(data.float()) 
        preds = output.cpu().data.max(1, keepdim=True)[1]
        test_preds = torch.cat((test_preds, preds), dim=0)
        
    return test_predictions

In [ ]:
test_set_preds = make_predictions(test_loader)

In [ ]:
submission_df=pd.DataFrame()
submission_df["label"]=test_set_preds.numpy().squeeze()
submission_df.head()

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = mnist["target"][60000:].astype(int)
y_predictions = np.array(submission_df["label"])
confusion_matrix(y_true, y_pred)